In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

In [ ]:
import configparser, networkx as nx, itertools, math, json, sys, time, timeit, numpy, pdb, jsbeautifier, re, spacy
from tqdm import tqdm


### Concept Grounding
Extract and match concepts from Ai2Thor objects list to ConceptNet concepts.

In [3]:
from grounding.grounding_concepts import GroundConcepts

In [4]:
config_path = Path("/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/paths.cfg")
ground = GroundConcepts(config_path)

In [5]:
ground.process('/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/objects.txt', )

Grounding: 100%|██████████| 87/87 [08:59<00:00,  6.20s/it]
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/odoemoo1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Pruning concepts: 100%|██████████| 87/87 [00:16<00:00,  5.28it/s]


Saved to /Users/odoemoo1/projects/visual-navigation/KagNet/datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json


### Generate Conceptnet graph

Run below in cell:
```
!cd ./pathfinder/`
!python graph_construction.py`
!cd ..
```

Outputs networkX graph of conceptnet.

### Path finding
Find a path between the concepts (i.e. objects).

Re-factored pathfinder.py to create an object. 

In [6]:
from pathfinder.pathfinder import PathFinder

In [8]:
config_fn = '/Users/odoemoo1/projects/visual-navigation/KagNet/pathfinder/paths.cfg'
pathfinder = PathFinder(config_fn)
fn = Path('./datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json')

loading conceptnet graph...
loading conceptnet graph...done


In [9]:
pathfinder.process(fn, nhops=1, beautify=True)

100%|██████████| 87/87 [00:00<00:00, 188.08it/s]


Found paths saved to datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts_1hops_paths.json


#### score paths
For concept and relation embeddings using glove embedding. In KagNet paper used knowledge graph embedding derived from TransE model. I will need to train this model to get the weights. For now use Glove embedding. 

In [11]:
from pathfinder.path_scoring import ScorePaths

In [30]:
scorer = ScorePaths(config_fn)
paths_fn = 'datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts_1hops_paths.json.pickle'
concepts_fn = 'datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json'
scorer.process(paths_fn, concepts_fn)

Scoring the paths:   0%|          | 0/87 [00:00<?, ?it/s]

datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.mean.npy file exists! Loading.


Pruning the paths: 100%|██████████| 87/87 [00:00<00:00, 5789.65it/s]

Scores saved to datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts_1hops_paths.json_scores.pickle
Num paths: 991, 		After Pruning: 991, keep rate: 1.0


In [ ]:
%debug

In [ ]:
from dgl import DGLGraph

In [ ]:
dg = DGLGraph()
dg.from_networkx(cpnet_simple)

In [ ]:
cpnet_simple.number_of_nodes()

In [ ]:
dg.subgraph([10, 20, 500]).adjacency_matrix()

TODO:
    - can you get a subgraph with just the nodes of AI2thor objects as concepts, without pathfinding? 